In [1]:
from sqlalchemy import create_engine
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from generate_plots_report import DataExtractor


sns.set_theme()

In [17]:
data_extractor = DataExtractor(db_connection, DAYS_FOR_REPORT)

In [18]:
article_topics_df = data_extractor.get_article_topics_df()
article_df = data_extractor.get_article_df()
feed_df = data_extractor.get_feed_df()
language_df = data_extractor.get_language_df()
bookmark_df = data_extractor.get_bookmark_df()
data_extractor.add_stats_to_feed(feed_df, article_df)

Getting Article Topics...
Getting Articles...
Getting feeds...
Getting Languages...
Getting bookmarks...


In [19]:
user_reading_time_df = data_extractor.get_user_reading_activity(
    language_df, feed_df
)

Getting user activity...


In [27]:
len(article_df[article_df.id.isin(user_reading_time_df.id)])

77

In [20]:
user_reading_time_df

,id,language_id,feed_id,user_id,total_reading_time,Language,Feed Name
0,2493085,3,NaN,4486,6.400000,German,No Feed
1,2493213,2,NaN,4590,5.350000,Danish,No Feed
2,2493357,3,76.0,4521,41.966667,German,76 Frankfurter All
3,2493606,2,NaN,4591,3.400000,Danish,No Feed
4,2493621,9,87.0,528,4.183333,Dutch,87 NRC
...,...,...,...,...,...,...,...
105,2504948,2,160.0,4022,0.933333,Danish,160 videnskab
106,2504939,2,145.0,4022,0.033333,Danish,145 Sport | DR
107,2499157,2,NaN,4022,0.083333,Danish,No Feed
108,2505478,2,161.0,4022,0.266667,Danish,161 bt


In [37]:
user_reading_time_df.Language.value_counts().reset_index().sort_values("count")

,Language,count
4,English,1
3,Dutch,3
2,German,17
1,French,30
0,Danish,59


In [30]:
article_df

,id,title,authors,content,htmlContent,summary,word_count,published_time,fk_difficulty,feed_id,url_id,language_id,broken,uploader_id,video,deleted,img_url_id,Language,Feed Name
0,2493023,Tientallen Palestijnen gedood bij Israëlische ...,Redactie Trouw,00:40\n\nHamas heeft Palestijnen zondag opgero...,"<div class=""page"" id=""readability-page-1""><div...",00:40\n\nHamas heeft Palestijnen zondag opgero...,8635,2024-05-27 01:27:31,70,85,2539618,9,0,None,0,0,2539619.0,Dutch,85 Trouw - Filosof
1,2493024,Geen slachtoffers gevonden na instorten parkee...,Marten Van De Wier,Nieuwegein\n\nZondagavond stortte een deel van...,"<div class=""page"" id=""readability-page-1""><div...",Nieuwegein\n\nZondagavond stortte een deel van...,448,2024-05-27 01:27:31,58,85,2539620,9,0,None,0,0,2539621.0,Dutch,85 Trouw - Filosof
2,2493025,Geen slachtoffers na instorten parkeergarage b...,,"NOS Nieuws•vandaag, 02:59\n\nEr is niemand gew...","<div class=""page"" id=""readability-page-1""><div...",Er is niemand gewond geraakt bij het instorten...,334,2024-05-27 01:27:39,56,89,2539622,9,0,None,0,0,2539623.0,Dutch,89 NOS Nieuws
3,2493026,Meerdere agenten gewond na ongeregeldheden na ...,,"NOS Nieuws•vandaag, 00:48\n\nDoor ongeregeldhe...","<div class=""page"" id=""readability-page-1""><div...",Door ongeregeldheden rond de wedstrijd van FC ...,181,2024-05-27 01:27:39,55,89,2539624,9,0,None,0,0,2539625.0,Dutch,89 NOS Nieuws
4,2493027,"Tientallen doden in tentenkamp Rafah, Israël z...",,"NOS Nieuws•gisteren, 23:34\n\nIn een tentenkam...","<div class=""page"" id=""readability-page-1""><div...",In een tentenkamp voor ontheemden in Rafah woe...,381,2024-05-27 01:27:39,66,89,2539626,9,0,None,0,0,2539627.0,Dutch,89 NOS Nieuws
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13651,2506913,The UK doesn’t work for Disabled people. Neith...,,“This election is about who our country works ...,"<div class=""page"" id=""readability-page-1""><div...","So far, we’ve heard more about Starmer and Sun...",809,2024-06-24 07:56:40,45,122,2566348,5,0,None,0,0,2566349.0,English,122 openDemocracy
13652,2506914,How to Vet Information Before Making a Decision,,Four questions to ask when you’re considering ...,"<div class=""page"" id=""readability-page-1""><div...",Four questions to ask when you’re considering ...,311,2024-06-24 07:56:43,43,126,2566350,5,0,None,0,0,2566351.0,English,126 HBR.org
13653,2506915,"When It Comes to Long-Term Value, Incumbents S...",,Six steps to kickstart your company’s “digital...,"<div class=""page"" id=""readability-page-1""><div...",Six steps to kickstart your company’s “digital...,471,2024-06-24 07:56:43,53,126,2566352,5,0,None,0,0,2566353.0,English,126 HBR.org
13654,2506916,Research: Warehouse and Logistics Automation W...,,A recent study suggests that blending human la...,"<div class=""page"" id=""readability-page-1""><div...",A recent study suggests that blending human la...,205,2024-06-24 07:56:43,60,126,2566354,5,0,None,0,0,2566355.0,English,126 HBR.org


In [31]:
top_5_articles_by_opened = user_reading_time_df.groupby(["Language", "Feed Name"]).id.value_counts().reset_index().sort_values("count", ascending=False)[:5]

In [33]:
top_5_articles_by_opened.merge(article_df[["id","title"]], on="id")[["Language", "Feed Name", "id", "title", "count"]]

,Language,Feed Name,id,title,count
0,Danish,160 videnskab,2494006,Kunstig intelligens undersøger effekten af mis...,3
1,French,60 Le Figaro - Act,2496904,"Albanie : à l’assaut des Alpes dinariques, par...",2
2,French,80 L'Equipe,2499838,Le Trophée des champions 2025 en Afrique,2
3,French,159 Le HuffPost : a,2499467,"Lors de France-Luxembourg, cette drôle d’illus...",2
4,Danish,136 Politiken.dk -,2493588,Sjælden fugl får unger i Danmark for første ga...,1


In [68]:
from nltk.tokenize import sent_tokenize
from collections import Counter
from tqdm import tqdm

def normalize_sent(text: str):
    return text.lower().strip()

def sent_count(text: str):
    return Counter([normalize_sent(sent) for paragraph in text.split("\n\n") for sent in sent_tokenize(paragraph) if len(sent.strip()) > 10 ])

def get_total_sent_counts(text_list: list[str]):
    total_counts = Counter()
    for text in tqdm(text_list, total=len(text_list)):
        total_counts += sent_count(text)
    return total_counts

In [ ]:
total_sent_count = get_total_sent_counts(last_week_articles.content)

100%|██████████| 1135/1135 [00:02<00:00, 461.98it/s]


In [ ]:
total_sent_count

Counter({'18 jun.video': 34,
         '17 jun.video': 22,
         'saltar recomendamos y continuar leyendo': 19,
         'recomendamos': 19,
         'final de recomendamos': 19,
         'blijf op de hoogte': 18,
         'haz clic aquí para leer más historias de bbc news mundo.': 18,
         'y recuerda que puedes recibir notificaciones en nuestra app.': 18,
         'descarga la última versión y actívalas.': 17,
         '18 jun.binnenland': 16,
         'ongeldig e-mailadres.': 14,
         'vul nogmaals in aub.': 14,
         'lees\xa0hier\xa0ons privacybeleid.': 14,
         'bekijk meer van': 14,
         'wil je op de hoogte blijven over alles rondom oorlog in oekraïne,\n      schrijf je dan in voor onze blijf op de hoogte nieuwsbrief.': 13,
         'veilig betalen': 13,
         '14 dagen bedenktijd': 13,
         '€ 79.99€ 189.95': 13,
         '€ 139.99€ 229.95': 13,
         '€ 54.99€ 99.99': 13,
         '€ 24.99€ 79.95': 13,
         'meer van de telegraaf webshop': 1

In [ ]:
new_sents = total_sent_count - previous_counter
new_sents

Counter({'18 jun.video': 34,
         '17 jun.video': 22,
         'saltar recomendamos y continuar leyendo': 19,
         'recomendamos': 19,
         'final de recomendamos': 19,
         'blijf op de hoogte': 18,
         'haz clic aquí para leer más historias de bbc news mundo.': 18,
         'y recuerda que puedes recibir notificaciones en nuestra app.': 18,
         'descarga la última versión y actívalas.': 17,
         '18 jun.binnenland': 16,
         'ongeldig e-mailadres.': 14,
         'vul nogmaals in aub.': 14,
         'lees\xa0hier\xa0ons privacybeleid.': 14,
         'bekijk meer van': 14,
         'wil je op de hoogte blijven over alles rondom oorlog in oekraïne,\n      schrijf je dan in voor onze blijf op de hoogte nieuwsbrief.': 13,
         'veilig betalen': 13,
         '14 dagen bedenktijd': 13,
         '€ 79.99€ 189.95': 13,
         '€ 139.99€ 229.95': 13,
         '€ 54.99€ 99.99': 13,
         '€ 24.99€ 79.95': 13,
         'meer van de telegraaf webshop': 1

In [ ]:
import os
import json

In [ ]:
data_loaded = {}
for file in os.listdir("report_data"):
    with open(os.path.join("report_data", file), "r", encoding="utf-8") as f:
        data_loaded[file] = json.load(f)

In [ ]:
data_loaded.keys()

dict_keys(['da-crawl-19_06_24_06_56_30.json', 'da-crawl-19_06_24_12_11_15.json', 'da-crawl-19_06_24_14_14_52.json', 'da-crawl-20_06_24_05_52_55.json', 'de-crawl-19_06_24_07_19_27.json', 'de-crawl-19_06_24_12_33_29.json', 'de-crawl-20_06_24_06_03_42.json', 'en-crawl-19_06_24_08_17_02.json', 'en-crawl-20_06_24_08_58_37.json', 'es-crawl-18_06_24_10_46_20.json', 'es-crawl-20_06_24_10_43_21.json', 'fr-crawl-19_06_24_07_16_52.json', 'fr-crawl-19_06_24_12_25_43.json', 'fr-crawl-20_06_24_09_46_58.json', 'nl-crawl-19_06_24_10_04_31.json', 'nl-crawl-20_06_24_08_42_14.json'])

In [ ]:
data_loaded['da-crawl-19_06_24_06_56_30.json']["feeds"]

{'82': {'article_data': {'sents_removed': {},
   'topics_counter': {'Culture': 3}},
  'last_crawled_time': '2024-06-19 06:51:25.256570',
  'repeated_articles': 7,
  'total_articles': 10,
  'total_downloaded': 3,
  'total_low_quality': 0,
  'total_in_db': 1,
  'crawl_time': 13.71},
 '135': {'article_data': {},
  'total_articles': 0,
  'total_downloaded': 0,
  'total_low_quality': 0,
  'total_in_db': 0,
  'crawl_time': 1.4},
 '136': {'article_data': {'sents_removed': {'Der er ikke oplæsning af denne artikel, så den oplæses derfor med maskinstemme.': 1,
    'Kontakt os gerne på automatiskoplaesning@pol.dk, hvis du hører ord, hvis udtale kan forbedres.': 1,
    '\n          Læs artiklen senere\n          Gemt (klik for at fjerne)\n          Læst': 2,
    '\n        FOR ABONNENTER': 2,
    'Prøv Politiken nu': 1},
   'topics_counter': {'Politics': 16, 'Culture': 2, 'Health': 1, 'Sport': 2},
   'quality_error': {'TOO_SHORT': 4}},
  'last_crawled_time': '2024-06-19 06:51:40.222930',
  'total_

In [ ]:
from collections import Counter

def accumulate_date_for_language(lang, loaded_data):
    accumulated_dict = {}
    accumulated_dict["feeds"] = {}
    for key in loaded_data.keys():
        current_dict = loaded_data[key]
        if lang in key:
            print(f"Reading... '{key}'")
            accumulated_dict["total_time"] = accumulated_dict.get("total_time", []) + [current_dict["total_time"]]
            accumulated_dict["date"] = accumulated_dict.get("date", []) + [key.split("-")[-1]]
            for feed in current_dict["feeds"].keys():
                if feed not in accumulated_dict["feeds"]:
                    accumulated_dict["feeds"][feed] = {}
                for key, val in current_dict["feeds"][feed].items():
                    if key == "article_data":
                        if key not in accumulated_dict["feeds"][feed]:
                            accumulated_dict["feeds"][feed][key] = {}
                        for art_key, art_val in current_dict["feeds"][feed]["article_data"].items():
                            accumulated_dict["feeds"][feed][key][art_key] = Counter(accumulated_dict["feeds"][feed].get(key, {}).get(art_key, {})) + Counter(art_val)
                    else:
                        accumulated_dict["feeds"][feed][key] = accumulated_dict["feeds"][feed].get(key, []) + [val]
    return accumulated_dict


In [ ]:
data_loaded['da-crawl-19_06_24_06_56_30.json'].keys()

dict_keys(['feeds', 'total_time'])

In [ ]:
acc_dict = accumulate_date_for_language("", data_loaded)

Reading... 'da-crawl-19_06_24_06_56_30.json'
Reading... 'da-crawl-19_06_24_12_11_15.json'
Reading... 'da-crawl-19_06_24_14_14_52.json'
Reading... 'da-crawl-20_06_24_05_52_55.json'
Reading... 'de-crawl-19_06_24_07_19_27.json'
Reading... 'de-crawl-19_06_24_12_33_29.json'
Reading... 'de-crawl-20_06_24_06_03_42.json'
Reading... 'en-crawl-19_06_24_08_17_02.json'
Reading... 'en-crawl-20_06_24_08_58_37.json'
Reading... 'es-crawl-18_06_24_10_46_20.json'
Reading... 'es-crawl-20_06_24_10_43_21.json'
Reading... 'fr-crawl-19_06_24_07_16_52.json'
Reading... 'fr-crawl-19_06_24_12_25_43.json'
Reading... 'fr-crawl-20_06_24_09_46_58.json'
Reading... 'nl-crawl-19_06_24_10_04_31.json'
Reading... 'nl-crawl-20_06_24_08_42_14.json'


In [ ]:
acc_dict["feeds"].keys()

dict_keys(['82', '135', '136', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '160', '161', '4', '15', '71', '76', '123', '124', '92', '94', '96', '97', '102', '103', '109', '110', '111', '112', '113', '114', '115', '122', '126', '127', '128', '129', '75', '95', '101', '104', '59', '60', '61', '66', '80', '83', '84', '107', '108', '130', '159', '41', '46', '50', '85', '87', '88', '89', '91', '125'])

In [ ]:
acc_dict["feeds"]

{'82': {'article_data': {'sents_removed': Counter(),
   'topics_counter': Counter({'Culture': 8})},
  'last_crawled_time': ['2024-06-19 06:51:25.256570',
   '2024-06-19 12:06:36.600204',
   '2024-06-19 14:03:18.854455',
   '2024-06-20 05:43:01.907458'],
  'repeated_articles': [7, 7, 10, 8],
  'total_articles': [10, 10, 10, 10],
  'total_downloaded': [3, 3, 0, 2],
  'total_low_quality': [0, 0, 0, 0],
  'total_in_db': [1, 1, 1, 1],
  'crawl_time': [13.71, 13.47, 0.94, 14.99]},
 '135': {'article_data': {},
  'total_articles': [0, 0, 0, 0],
  'total_downloaded': [0, 0, 0, 0],
  'total_low_quality': [0, 0, 0, 0],
  'total_in_db': [0, 0, 0, 0],
  'crawl_time': [1.4, 1.43, 1.31, 1.64]},
 '136': {'article_data': {'sents_removed': Counter({'\n          Læs artiklen senere\n          Gemt (klik for at fjerne)\n          Læst': 4,
            '\n        FOR ABONNENTER': 4,
            'Der er ikke oplæsning af denne artikel, så den oplæses derfor med maskinstemme.': 2,
            'Kontakt os ger

In [ ]:
def get_total_quality_counts(acc_dict):
    total_counts = Counter()
    for feed in acc_dict["feeds"]:
        if "article_data" not in acc_dict["feeds"][feed]:
            continue
        if "quality_error" not in acc_dict["feeds"][feed]["article_data"]:
            continue
        total_counts += Counter(acc_dict["feeds"][feed]["article_data"]["quality_error"])
    return total_counts

In [ ]:
total_rejected_article_reasons = dict(get_total_quality_counts(acc_dict))

In [ ]:
total_rejected_article_reasons
total_rejected_article_reasons["Total"] = sum(total_rejected_article_reasons.values())

In [ ]:
pd_quality_errors = pd.DataFrame.from_dict(total_rejected_article_reasons, orient="index").reset_index()
pd_quality_errors.columns = ["Reason", "Count"]

In [ ]:
print(pd_quality_errors.to_html(index=False))

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Reason</th>
      <th>Count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>TOO_SHORT</td>
      <td>485</td>
    </tr>
    <tr>
      <td>ML_PREDICTION</td>
      <td>165</td>
    </tr>
    <tr>
      <td>HTML_PATTERN</td>
      <td>122</td>
    </tr>
    <tr>
      <td>Total</td>
      <td>772</td>
    </tr>
  </tbody>
</table>


In [685]:
previous_counter = total_sent_count